# Travelling Salesman Problem
## Hill-Climbing + Genetic Algorithm
### Final Expanded Notebook (No File Saving)

This notebook contains the full implementation of:
- Hill Climbing with Random Restarts
- Genetic Algorithm (GA)
- Comparative Analysis
- Multiple charts and tables


In [ ]:

import numpy as np
import pandas as pd
import math
import random
import time
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

def euclidean(a,b):
    return math.hypot(a[0]-b[0], a[1]-b[1])

def route_distance(route, coords):
    dist = 0
    for i in range(len(route)):
        a = coords[route[i]]
        b = coords[(i+1) % len(route)]
        dist += euclidean(a,b)
    return dist

def plot_route(route, coords, title='Route'):
    xs = [coords[i][0] for i in route] + [coords[route[0]][0]]
    ys = [coords[i][1] for i in route] + [coords[route[0]][1]]
    plt.figure(figsize=(6,6))
    plt.plot(xs, ys, '-o')
    for i,(x,y) in enumerate(coords):
        plt.text(x,y,str(i))
    plt.title(title)
    plt.show()


In [ ]:

df = pd.read_csv('/mnt/data/cities.csv')
df.columns = [c.lower() for c in df.columns]
coords = df[['x','y']].values.tolist() if 'x' in df.columns else df.iloc[:,:2].values.tolist()
print('Loaded', len(coords), 'cities')
df.head()


## Hill Climbing Implementation

In [ ]:

def random_route(n):
    r=list(range(n))
    random.shuffle(r)
    return r

def swap_neighbors(route):
    n=len(route)
    for i in range(n-1):
        for j in range(i+1,n):
            r=route.copy()
            r[i],r[j]=r[j],r[i]
            yield r

def hill_climb(start, coords, max_iter=1000):
    curr=start.copy()
    curr_dist=route_distance(curr,coords)
    history=[curr_dist]
    for _ in range(max_iter):
        improved=False
        best=curr_dist
        best_route=None
        for neigh in swap_neighbors(curr):
            d=route_distance(neigh,coords)
            if d<best:
                best=d
                best_route=neigh
        if best_route:
            curr=best_route
            curr_dist=best
            history.append(curr_dist)
            improved=True
        if not improved:
            break
    return curr,curr_dist,history

def hillclimb_restart(coords, restarts=40):
    best=None
    best_dist=float('inf')
    histories=[]
    t0=time.time()
    for _ in range(restarts):
        start=random_route(len(coords))
        rt,d,h=hill_climb(start,coords)
        histories.append(h)
        if d<best_dist:
            best_dist=d
            best=rt
    return best,best_dist,histories,time.time()-t0

hillclimb_restart(coords,5)[1]


## Genetic Algorithm Implementation

In [ ]:

def order_crossover(p1,p2):
    n=len(p1)
    a,b=sorted(random.sample(range(n),2))
    child=[-1]*n
    child[a:b+1]=p1[a:b+1]
    idx=0
    for i in range(n):
        if child[i]==-1:
            while p2[idx] in child:
                idx+=1
            child[i]=p2[idx]
            idx+=1
    return child

def swap_mut(route, rate=0.03):
    r=route.copy()
    for i in range(len(r)):
        if random.random()<rate:
            j=random.randrange(len(r))
            r[i],r[j]=r[j],r[i]
    return r

def tournament(pop,fits,k=3):
    cand=random.sample(range(len(pop)),k)
    cand.sort(key=lambda x:fits[x])
    return pop[cand[0]].copy()

def GA(coords, pop_size=120, gens=250, mrate=0.03):
    pop=[random_route(len(coords)) for _ in range(pop_size)]
    fits=[route_distance(r,coords) for r in pop]
    best=min(fits)
    bestroute=pop[fits.index(best)].copy()
    history=[best]
    t0=time.time()

    for _ in range(gens):
        new=[]
        while len(new)<pop_size:
            p1=tournament(pop,fits)
            p2=tournament(pop,fits)
            child=order_crossover(p1,p2) if random.random()<0.9 else p1.copy()
            child=swap_mut(child,mrate)
            new.append(child)
        pop=new
        fits=[route_distance(r,coords) for r in pop]
        curr=min(fits)
        if curr<best:
            best=curr
            bestroute=pop[fits.index(curr)].copy()
        history.append(best)
    return bestroute,best,history,time.time()-t0

GA(coords,60,100)[1]


## Experiments

In [ ]:

sizes=[10,20,30,40,50]
trials=5
results=[]

for s in sizes:
    sub=coords[:s]
    print('===',s,'cities ===')
    for t in range(trials):
        _,d,_,tm=hillclimb_restart(sub,40)
        results.append([s,'HillClimb',t,d,tm])
        print(f"HC[{t}] dist={d:.2f} time={tm:.2f}", end=" | ")
    print()
    for t in range(trials):
        _,d,_,tm=GA(sub,120,250,0.03)
        results.append([s,'GA',t,d,tm])
        print(f"GA[{t}] dist={d:.2f} time={tm:.2f}", end=" | ")
    print()

df_res=pd.DataFrame(results,columns=['size','algo','trial','distance','time'])
df_res


## Charts

In [ ]:

plt.figure(figsize=(10,5))
sns.boxplot(data=df_res,x='size',y='distance',hue='algo')
plt.title('Distance Comparison')
plt.show()

plt.figure(figsize=(10,5))
sns.boxplot(data=df_res,x='size',y='time',hue='algo')
plt.title('Runtime Comparison')
plt.show()


In [ ]:

avg=df_res.groupby(['size','algo'])['distance'].mean().reset_index()
plt.figure(figsize=(8,5))
sns.lineplot(data=avg,x='size',y='distance',hue='algo',marker='o')
plt.title('Mean Distance vs Problem Size')
plt.show()

avgt=df_res.groupby(['size','algo'])['time'].mean().reset_index()
plt.figure(figsize=(8,5))
sns.lineplot(data=avgt,x='size',y='time',hue='algo',marker='o')
plt.title('Mean Runtime vs Problem Size')
plt.show()


## Comparison Tables

In [ ]:

dist_table=df_res.groupby(['size','algo'])['distance'].agg(['mean','median','std'])
time_table=df_res.groupby(['size','algo'])['time'].agg(['mean','median','std'])
dist_table, time_table
